In [1]:
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline
from sklearn.preprocessing import StandardScaler
from scipy.stats import ttest_ind
import seaborn as sns
import numpy as np
from scipy import stats
from sklearn.metrics import mean_squared_error
import pmdarima
from pmdarima import auto_arima
import time
import plotly.graph_objects as go
import pandas as pd
import statsmodels.api as sm
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
from multiprocesspandas import applyparallel

# ARIMA on per item sales level

In [2]:
orig = pd.read_csv(r"data\train.csv", dtype={'id': 'object', 'item_id': 'category', 'subcat_id': 'category', 'category_id': 'category', 'store_id':'category', 'region_id':'category'})
sample = pd.read_csv(r'data\sample_submission.csv')

In [ ]:
orig.head()
orig = orig.drop(columns=['item_id', 'subcat_id', 'category_id', 'store_id', 'region_id'])

In [ ]:
orig_long = orig.transpose()

In [ ]:
orig_long.columns = orig_long.iloc[0]
orig_long = orig_long.drop(index='id')

In [ ]:
del(orig)

In [6]:
def auto_arima_column_mp(col, **kwargs):
    import pmdarima
    # print(col)
    first_day_of_sales = (col != 0).idxmax()
    cropped_data = col.loc[first_day_of_sales:]
    model_spec = pmdarima.arima.ARIMA(**kwargs)
    pred = model_spec.fit_predict(cropped_data.values, X=None, n_periods=21)
    # print(col)
    return pred

def auto_arima_all_column_pb_md(df, sample, start=0, end=30490, **kwargs):
    df= df.loc[:, df.columns[start:end]]
    pred = df.apply_parallel(auto_arima_column_mp, axis = 1, num_processors=4, **kwargs)
    print(pred)
    for i in range(start,end):
        sample.iloc[i,1:] = pred[i]
    return sample

In [7]:
t7 = auto_arima_all_column_pb_md(orig_long, sample, start=0, end=30490, order = (0,1,1), seasonal_order=(0, 1, 1, 7), maxiter=25, suppress_warnings=False, scoring='mse', scoring_args=None, with_intercept=False)

100%|██████████| 1000/1000 [00:11<00:00, 89.98it/s]


In [ ]:
t7

,id,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,...,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940
0,Beauty_1_001_East_1,1.201507,0.676732,1.292862,1.177819,1.668387,1.482700,1.458557,1.699419,0.791221,...,1.654774,1.110728,1.751050,1.647198,0.922374,1.343315,1.111583,1.633604,0.867423,1.684015
1,Beauty_1_002_East_1,0.087217,0.091235,-0.030516,-0.038649,0.082063,-0.004155,0.082063,0.005039,0.005463,...,0.027504,0.018415,0.027504,0.027868,0.028304,-0.009539,0.009359,-0.038828,-0.048174,-0.038828
2,Beauty_1_003_East_1,0.935931,0.472100,0.571261,0.702490,0.994022,0.956689,1.039012,1.138129,0.586875,...,0.980635,1.185872,1.025777,1.104839,0.588026,0.593905,0.975114,1.118011,1.123860,1.300576
3,Beauty_1_004_East_1,3.202852,2.653210,1.037733,1.393890,1.810590,2.209661,3.580977,3.389968,2.155425,...,1.878792,2.535688,3.128061,3.591594,2.074991,1.152168,1.349062,1.505316,2.477220,3.726340
4,Beauty_1_005_East_1,1.414597,1.361649,1.006001,1.300172,1.171689,1.531302,1.144793,1.842896,1.159815,...,1.657446,1.080098,0.964651,1.978056,1.045745,0.788134,1.634473,1.926316,0.883234,0.883076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,Food_3_823_West_3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
30486,Food_3_824_West_3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
30487,Food_3_825_West_3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
30488,Food_3_826_West_3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# t7.to_csv("Sean 4.csv", index=False)